<a href="https://colab.research.google.com/github/codingiamnot/PMP-2024/blob/main/Lab12/pmp_lab12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt

data = pd.read_csv("date_promovare_examen.csv")

hStudy = np.array(data["Ore_Studiu"])
hSleep = np.array(data["Ore_Somn"])
promovare = np.array(data["Promovare"])

In [19]:
with pm.Model() as model:
  coefStudy = pm.Normal('coefStudy', mu=0, sigma=200)
  coefSleep = pm.Normal('coefSleep', mu=0, sigma=200)
  bias = pm.Normal('bias', mu=0, sigma=1000)

  val = coefStudy * hStudy + coefSleep * hSleep + bias
  p = pm.Deterministic('p', pm.math.sigmoid(val))

  rez = pm.Bernoulli('rez', p=p, observed=promovare)

  sample = pm.sample(2000, return_inferencedata=True)

Output()

Output()

ERROR:pymc.stats.convergence:There were 2834 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [20]:
posterior_g = sample.posterior.stack(samples={"chain", "draw"})

ansCoefStudy = posterior_g['coefStudy'].mean().item()
ansCoefSleep = posterior_g['coefSleep'].mean().item()
ansBias = posterior_g['bias'].mean().item()

print(ansCoefStudy, ansCoefSleep, ansBias)

2.0628722496864085 7.447982876536033 -56.4185554661013


b) Granita este data de valorile $coefStudy$, $coefSleep$ si $bias$.
Pentru o instanta conditia este : $hStudy * coefStudy + hSleep * coefSleep + bias \geq 0$. In cazul nostru: $hStudy * 2.06.. + hSleep * 7.44.. -56.41.. \geq 0$

In [21]:
corectGuess = 0

for i in range(len(promovare)):
  val = hStudy[i] * ansCoefStudy + hSleep[i] * ansCoefSleep + ansBias

  if(val > 0 and promovare[i] == 1):
    corectGuess += 1

  elif(val < 0 and promovare[i] == 0):
    corectGuess += 1


print(corectGuess / len(promovare))

1.0


Deoarece modelul are acuratete 100% inseamna ca datele sunt perfect separabile.

c) Din model rezulta ca orele de somn sunt mai importante decat cele de studiu.
$coefSleep \geq coefStudy$ $(7.44 \geq 2.06)$